In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [32]:
url="http://www.wblivesurf.com/reports/c-street-2257/" 
r=requests.get(url)
soup=BeautifulSoup(r.content,'html.parser')

In [33]:
print(soup.prettify())

<!DOCTYPE doctype html>
<html>
 <head>
  <meta charset="utf-8"/>
  <title>
   C-Street - WB Live Surf
  </title>
  <meta content="C-Street - WB Live Surf" name="title"/>
  <meta content="Copyright © WB Live Surf 2019. All Rights Reserved." name="Copyright"/>
  <meta content="width=device-width, initial-scale=1.0 minimal-ui" name="viewport">
   <link href="http://www.wblivesurf.com/wp-content/themes/WBLiveSurf/img/favicon.ico" rel="shortcut icon"/>
   <link href="http://www.wblivesurf.com/wp-content/themes/WBLiveSurf/reset.css" rel="stylesheet"/>
   <link href="http://www.wblivesurf.com/wp-content/themes/WBLiveSurf/style.css" rel="stylesheet"/>
   <link href="http://www.wblivesurf.com/xmlrpc.php" rel="pingback"/>
   <!-- This site is optimized with the Yoast SEO plugin v12.5 - https://yoast.com/wordpress/plugins/seo/ -->
   <meta content="max-snippet:-1, max-image-preview:large, max-video-preview:-1" name="robots">
    <link href="http://www.wblivesurf.com/reports/c-street-27/" rel="can

In [34]:
conditions={}

for strong_tag in soup.find_all('strong'):
    conditions.update({strong_tag.text : strong_tag.next_sibling})

In [35]:
conditions

{'Lunchtime Update:\xa0 ': "It's looking even better now. Chest to head high sets and open barrels on the better ones. The rights are longer and more open, but the lefts are open, shorter, and a little easier to make. Either way you go it's fun and bigger than average. I would have given it 4 logos, but lots of the guys out front were really working hard to get into the wave. The temp has dropped almost 20 degrees since this morning so be prepared for that. Have fun if you go.",
 'Swell Size:': ' Chest/Shoulder/Occ. Head High',
 'Water Surface:': ' Semi-Choppy/Clean Faces',
 'Water Temp:': ' 61',
 'Wind:': ' WSW 4-8 Plus',
 'High Tide:': ' 1:57:00 PM',
 'Low Tide:': ' 8:38:00 PM',
 'Sunrise:': ' 6:55:00 AM',
 'Sunset:': ' 5:02:00 PM',
 'Sweetwater': ' Marketplace',
 'surfreports': None,
 'WB Live Surf': ' Marketplace',
 'Local': ' Forecast'}

In [36]:
size=conditions['Swell Size:'].lstrip()
surface=conditions['Water Surface:'].lstrip()
temp=conditions['Water Temp:'].lstrip()
wind=conditions['Wind:'].lstrip()
high=conditions['High Tide:'].lstrip()
low=conditions['Low Tide:'].lstrip()
rise=conditions['Sunrise:'].lstrip()
sunset=conditions['Sunset:'].lstrip()

In [37]:
report=soup.find('div',attrs={'class':'entry-content'})

In [38]:
report=str(report)
report

'<div class="entry-content">\n<a href="http://www.wblivesurf.com/reports/c-street-27/" rel="bookmark" title="12:36 pm"><time class="entry-date" datetime="2013-11-27T12:36:00-05:00" pubdate="">November 27, 2013</time></a> <p><strong>Lunchtime Update:\xa0 </strong>It\'s looking even better now. Chest to head high sets and open barrels on the better ones. The rights are longer and more open, but the lefts are open, shorter, and a little easier to make. Either way you go it\'s fun and bigger than average. I would have given it 4 logos, but lots of the guys out front were really working hard to get into the wave. The temp has dropped almost 20 degrees since this morning so be prepared for that. Have fun if you go.</p>\n<p><img border="0" height="17" src="../images/rating_star.jpg" width="17"/>Surf Report Special- Mention the surf report at Sweetwater and get 20% off fall and winter clothing</p>\n<p><img border="0" height="17" src="../images/rating_star.jpg" width="17"/>Check out the amazing

In [39]:
pattern=r"\"(\d*\:\d.+?)\""
time=re.findall(pattern,report)

In [40]:
pattern=r"\"(\d\d\d\d\-\d.\-\d.?)T"
date=re.findall(pattern,report)

In [41]:
pattern=r"g\>([a-zA-Z].+)\:"
when=re.findall(pattern,report)

In [42]:
pattern="g\>.+g\>([a-zA-Z].+)\</"
text=re.findall(pattern,report)

In [43]:
report_dict={'date':date,'part_of_day':when,'time':time,'size':size,'surface':surface,'temp':temp,'wind':wind,
             'high':high,'low':low,'rise':rise,'sunset':sunset,'text':text}

In [44]:
report_df=pd.DataFrame(report_dict)

In [45]:
report_df

,date,part_of_day,time,size,surface,temp,wind,high,low,rise,sunset,text
0,2013-11-27,Lunchtime Update,12:36 pm,Chest/Shoulder/Occ. Head High,Semi-Choppy/Clean Faces,61,WSW 4-8 Plus,1:57:00 PM,8:38:00 PM,6:55:00 AM,5:02:00 PM,It's looking even better now. Chest to head hi...


In [17]:
report_df=pd.DataFrame(columns=['date','part_of_day','time','size','surface','temp','wind','high','low','rise',
                                'sunset','text'])

for x in range(10):
    x=1
    url="http://wblivesurf.com/reports/c-street-"+str(10+x)
    r=requests.get(url)
    soup=BeautifulSoup(r.content,'html.parser')

    error=soup.find('h2').text
    skip='Error 404 - Page Not Found'
    if error == skip:
        continue

    conditions={}
    for strong_tag in soup.find_all('strong'):
        conditions.update({strong_tag.text:strong_tag.next_sibling})
    
    size=conditions['Swell Size:'].lstrip()
    surface=conditions['Water Surface:'].lstrip()
    temp=conditions['Water Temp:'].lstrip()
    wind=conditions['Wind:'].lstrip()
    high=conditions['High Tide:'].lstrip()
    low=conditions['Low Tide:'].lstrip()
    rise=conditions['Sunrise:'].lstrip()
    sunset=conditions['Sunset:'].lstrip()

    report=soup.find('div',attrs={'class':'entry-content'})
    report=str(report)

    pattern=r"\"(\d*\:\d.+?)\""
    time=re.findall(pattern,report)

    pattern=r"\"(\d\d\d\d\-\d.\-\d.?)T"
    date=re.findall(pattern,report)

    pattern=r"g\>([a-zA-Z].+)\:"
    when=re.findall(pattern,report)

    pattern="g\>.+g\>([a-zA-Z].+)\</"
    text=re.findall(pattern,report)

    report_dict={'date':date[0],'part_of_day':when[0],'time':time[0],'size':size,'surface':surface,'temp':temp,'wind':wind,
             'high':high,'low':low,'rise':rise,'sunset':sunset,'text':text[0]}

    temp_df=pd.DataFrame(report_dict,index=[0])

    report_df=report_df.append(temp_df)
    break

         date        part_of_day      time              size      surface  \
0  2017-01-23  Lunch-time Update  12:32 pm  Waist-Chest High  Slight Bump   

   temp           wind        high          low        rise      sunset  \
0  55.6  SSW 5-10 Plus  4:04:00 PM  10:10:00 AM  7:13:00 AM  5:32:00 PM   

                                                text  
0  Still some fun looking surf rolling through.  ...  


In [18]:
report_df

,date,part_of_day,time,size,surface,temp,wind,high,low,rise,sunset,text
0,2017-01-23,Lunch-time Update,12:32 pm,Waist-Chest High,Slight Bump,55.6,SSW 5-10 Plus,4:04:00 PM,10:10:00 AM,7:13:00 AM,5:32:00 PM,Still some fun looking surf rolling through. ...


In [7]:
url

'http://wblivesurf.com/reports/c-street-11'